In [2]:
!pip install pandas requests beautifulsoup4 pdfminer.six lxml > /dev/null 2>&1

The system cannot find the path specified.


In [1]:
import argparse
import io
import os
import re
import time
import urllib
from concurrent.futures import ThreadPoolExecutor, wait
from datetime import date
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pdfminer import high_level

In [3]:
def create_path(folder_name):
    path = os.path.join(os.getcwd(), folder_name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [4]:
def open_page(link):
    count = 0
    while count < 3:
        try:
            return BeautifulSoup(requests.get(link).text, "lxml")
        except:
            count += 1
            time.sleep(5)

def get_detail(soup, keyword):
    try:
        text = (
            soup.find(lambda tag: tag.name == "td" and keyword in tag.text)
            .find_next()
            .get_text()
            .strip()
        )
        return text
    except:
        return ""

def get_pdf(url, path_pdf):
    file = urllib.request.urlopen(url)
    file_name = file.info().get_filename().replace("/", " ")
    file_content = file.read()
    with open(f"{path_pdf}/{file_name}", "wb") as out_file:
        out_file.write(file_content)
    return io.BytesIO(file_content), file_name

In [5]:
def clean_text(text):
    text = text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    text = text.replace("Disclaimer\n", "")
    text = text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        "",
    )
    text = text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        "",
    )
    text = text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n",
        "",
    )
    text = text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n",
        "",
    )
    return text

In [6]:
def is_url_already_scraped(url, destination):
    """
    Checks if a URL has already been scraped and saved in the CSV file.

    Args:
        url (str): The URL to check.
        destination (str): The path to the output CSV file.

    Returns:
        bool: True if the URL exists in the CSV, False otherwise.
    """
    if not os.path.isfile(f"{destination}.csv"):
      return False

    try:
        df = pd.read_csv(f"{destination}.csv")
        return url in df["link"].values
    except pd.errors.EmptyDataError:
      return False

In [7]:
def extract_data(link, keyword_url):
    global today
    global path_output
    global path_pdf
    global download_pdf

    path_output = 'data/raw/CSV'
    path_pdf = 'data/raw/PDF'
    today = date.today().strftime("%Y-%m-%d")

    keyword_url = keyword_url.replace("/", " ")
    if keyword_url.startswith("https"):
        keyword_url = ""
    destination = f"{path_output}/putusan_ma_{keyword_url}_{today}"

    if is_url_already_scraped(link, destination):
        print(f"Skipping duplicate URL: {link}")
        return

    soup = open_page(link)
    table = soup.find("table", {"class": "table"})
    judul = table.find("h2").text
    table.find("h2").decompose()

    nomor = get_detail(table, "Nomor")
    tingkat_proses = get_detail(table, "Tingkat Proses")
    klasifikasi = get_detail(table, "Klasifikasi")
    kata_kunci = get_detail(table, "Kata Kunci")
    tahun = get_detail(table, "Tahun")
    tanggal_register = get_detail(table, "Tanggal Register")
    lembaga_peradilan = get_detail(table, "Lembaga Peradilan")
    jenis_lembaga_peradilan = get_detail(table, "Jenis Lembaga Peradilan")
    hakim_ketua = get_detail(table, "Hakim Ketua")
    hakim_anggota = get_detail(table, "Hakim Anggota")
    panitera = get_detail(table, "Panitera")
    amar = get_detail(table, "Amar")
    amar_lainnya = get_detail(table, "Amar Lainnya")
    catatan_amar = get_detail(table, "Catatan Amar")
    tanggal_musyawarah = get_detail(table, "Tanggal Musyawarah")
    tanggal_dibacakan = get_detail(table, "Tanggal Dibacakan")
    kaidah = get_detail(table, "Kaidah")
    abstrak = get_detail(table, "Abstrak")

    try:
        link_pdf = soup.find("a", href=re.compile(r"/pdf/"))["href"]
        file_pdf, file_name_pdf = get_pdf(link_pdf, path_pdf)
        text_pdf = high_level.extract_text(file_pdf)
        text_pdf = clean_text(text_pdf)
    except:
        link_pdf = ""
        text_pdf = ""
        file_name_pdf = ""

    data = [
        judul,
        nomor,
        tingkat_proses,
        klasifikasi,
        kata_kunci,
        tahun,
        tanggal_register,
        lembaga_peradilan,
        jenis_lembaga_peradilan,
        hakim_ketua,
        hakim_anggota,
        panitera,
        amar,
        amar_lainnya,
        catatan_amar,
        tanggal_musyawarah,
        tanggal_dibacakan,
        kaidah,
        abstrak,
        link,
        link_pdf,
        file_name_pdf,
        text_pdf,
    ]
    result = pd.DataFrame(
        [data],
        columns=[
            "judul",
            "nomor",
            "tingkat_proses",
            "klasifikasi",
            "kata_kunci",
            "tahun",
            "tanggal_register",
            "lembaga_peradilan",
            "jenis_lembaga_peradilan",
            "hakim_ketua",
            "hakim_anggota",
            "panitera",
            "amar",
            "amar_lainnya",
            "catatan_amar",
            "tanggal_musyawarah",
            "tanggal_dibacakan",
            "kaidah",
            "abstrak",
            "link",
            "link_pdf",
            "file_name_pdf",
            "text_pdf",
        ],
    )

    print(destination)
    if not os.path.isfile(f"{destination}.csv"):
        result.to_csv(f"{destination}.csv", header=True, index=False)
    else:
        result.to_csv(f"{destination}.csv", mode="a", header=False, index=False)

In [8]:
def run_scraper(keyword=None, url=None, sort_date=True, download_pdf=True):
    if not keyword and not url:
        print("Please provide a keyword or URL")
        return

    path_output = 'data/raw/CSV'
    path_pdf = 'data/raw/PDF'

    today = date.today().strftime("%Y-%m-%d")

    link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword}&page=1"

    if url:
        link = url

    soup = open_page(link)

    last_page = int(
        soup.find_all("a", {"class": "page-link"})[-1].get("data-ci-pagination-page")
    )

    if url:
        print(f"Scraping with url: {url} - {20 * last_page} data - {last_page} page")
    else:
        print(f"Scraping with keyword: {keyword} - {20 * last_page} data - {last_page} page")

    if url:
        keyword_url = url
    else:
        keyword_url = keyword

    futures = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for page in range(last_page):
            futures.append(
                executor.submit(run_process, keyword_url, page + 1, sort_date)
            )
    wait(futures)

def run_process(keyword_url, page, sort_page):
    if keyword_url.startswith("https"):
        link = f"{keyword_url}&page={page}"
    else:
        link = f"https://putusan3.mahkamahagung.go.id/search.html?q={keyword_url}&page={page}"
    if sort_page:
        link = f"{link}&obf=TANGGAL_PUTUS&obm=desc"

    print(link)

    soup = open_page(link)
    links = soup.find_all("a", {"href": re.compile("/direktori/putusan")})

    for link in links:
        extract_data(link["href"], keyword_url)

def scrape_specific_url(url, download_pdf=True):
    if not url or not url.startswith("https://"):
        print("Please provide a valid URL")
        return

    path_output = 'data/raw/CSV'
    path_pdf = 'data/raw/PDF'
    today = date.today().strftime("%Y-%m-%d")

    extract_data(url, url, path_output, path_pdf, today)

In [9]:
# Penganiayaan PN MEDAN
run_scraper(url="https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr=")

Scraping with url: https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr= - 100 data - 5 page
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr=&page=1&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr=&page=2&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr=&page=3&obf=TANGGAL_PUTUS&obm=desc
https://putusan3.mahkamahagung.go.id/search.html?q=&jenis_doc=putusan&cat=4210395c07581866de499a7b125ea4bc&jd=&tp=0&court=098629PN185&t_put=2024&t_reg=&t_upl=&t_pr=&page=4&obf=TANGGAL_PUTUS&obm=desc


In [3]:
import os
import re
from pdfminer.high_level import extract_text
from datetime import datetime

In [16]:
# === Konfigurasi Path ===
PDF_DIR = 'data/raw/PDF'
TEXT_DIR = 'data/raw'
LOG_PATH = 'logs/cleaning.log'

# === Buat Folder jika Belum Ada ===
os.makedirs(TEXT_DIR, exist_ok=True)
os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)

In [19]:
# === Fungsi Pembersihan Teks ===
def clean_text(text):
    # Hapus watermark dan disclaimer tetap dari Mahkamah Agung
    text = text.replace("M a h ka m a h A g u n g R e p u blik In d o n esia\n", "")
    text = text.replace("Disclaimer\n", "")
    text = text.replace(
        "Kepaniteraan Mahkamah Agung Republik Indonesia berusaha untuk selalu mencantumkan informasi paling kini dan akurat sebagai bentuk komitmen Mahkamah Agung untuk pelayanan publik, transparansi dan akuntabilitas\n",
        ""
    )
    text = text.replace(
        "pelaksanaan fungsi peradilan. Namun dalam hal-hal tertentu masih dimungkinkan terjadi permasalahan teknis terkait dengan akurasi dan keterkinian informasi yang kami sajikan, hal mana akan terus kami perbaiki dari waktu kewaktu.\n",
        ""
    )
    text = text.replace(
        "Dalam hal Anda menemukan inakurasi informasi yang termuat pada situs ini atau informasi yang seharusnya ada, namun belum tersedia, maka harap segera hubungi Kepaniteraan Mahkamah Agung RI melalui :\n",
        ""
    )
    text = text.replace(
        "Email : kepaniteraan@mahkamahagung.go.id    Telp : 021-384 3348 (ext.318)\n",
        ""
    )

    # Hapus baris kosong berlebih, nomor halaman, dsb
    text = re.sub(r'\n\s*\n', '\n', text)
    text = re.sub(r'Halaman\s+\d+\s+dari\s+\d+', '', text, flags=re.I)

    # Normalisasi teks
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # hapus tanda baca (opsional)
    text = re.sub(r'\s+', ' ', text)     # normalisasi spasi

    return text.strip()

In [13]:
# === Fungsi Validasi Keutuhan Teks ===
def is_valid_text(text, min_words=500):
    return len(text.split()) >= min_words  # asumsikan ≥ 500 kata cukup utuh (~80%)

In [20]:
# === Proses Seluruh PDF ===
def convert_and_clean_all():
    log_lines = []
    for i, filename in enumerate(sorted(os.listdir(PDF_DIR))):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(PDF_DIR, filename)
            case_id = f"case_{i+1:03d}"
            txt_path = os.path.join(TEXT_DIR, f"{case_id}.txt")

            try:
                # Ekstraksi teks
                raw_text = extract_text(pdf_path)
                cleaned_text = clean_text(raw_text)
                valid = is_valid_text(cleaned_text)

                # Simpan hasil pembersihan
                with open(txt_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_text)

                # Log hasil
                log_line = f"[{datetime.now()}] {filename} → {case_id}.txt | Status: {'OK' if valid else 'INCOMPLETE'} | Words: {len(cleaned_text.split())}"
                print(log_line)
                log_lines.append(log_line)

            except Exception as e:
                error_line = f"[{datetime.now()}] {filename} → ERROR: {str(e)}"
                print(error_line)
                log_lines.append(error_line)

    # Simpan log
    with open(LOG_PATH, 'w', encoding='utf-8') as log_file:
        log_file.write("\n".join(log_lines))


# === Jalankan Proses ===
convert_and_clean_all()

[2025-06-25 18:30:33.320097] putusan_1006_pid.b_2024_pn_mdn_20250625174208.pdf → case_001.txt | Status: OK | Words: 3978
[2025-06-25 18:30:34.097868] putusan_100_pid.b_2024_pn_mdn_20250625174338.pdf → case_002.txt | Status: OK | Words: 4937
[2025-06-25 18:30:34.931940] putusan_101_pid.b_2024_pn_mdn_20250625174255.pdf → case_003.txt | Status: OK | Words: 5354
[2025-06-25 18:30:36.051612] putusan_1055_pid.b_2024_pn_mdn_20250625173949.pdf → case_004.txt | Status: OK | Words: 7577
[2025-06-25 18:30:36.816608] putusan_1063_pid.b_2024_pn_mdn_20250625174344.pdf → case_005.txt | Status: OK | Words: 5246
[2025-06-25 18:30:37.672147] putusan_1080_pid.b_2024_pn_mdn_20250625174440.pdf → case_006.txt | Status: OK | Words: 5566
[2025-06-25 18:30:38.202551] putusan_1101_pid.b_2024_pn_mdn_20250625174256.pdf → case_007.txt | Status: OK | Words: 3330
[2025-06-25 18:30:38.634406] putusan_1102_pid.b_2024_pn_mdn_20250625174621.pdf → case_008.txt | Status: OK | Words: 2827
[2025-06-25 18:30:39.438560] putus

In [4]:
import pandas as pd

In [160]:
# Load CSV dari local folder
file_path = 'data/raw/CSV/putusan_ma__2025-06-25.csv'
df = pd.read_csv(file_path)

# Tampilkan data awal
df.head()

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,...,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,link,link_pdf,file_name_pdf,text_pdf
0,Putusan PN MEDAN Nomor 621/Pid.B/2024/PN Mdn T...,621/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,2 Mei 2024,PN MEDAN,PN,Hakim Ketua Lucas Sahabat Duha,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I Menyatakan Terdakwa I. M. Ka...,23 Juli 2024,23 Juli 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_621_pid.b_2024_pn_mdn_20250625173548.pdf,Direktori Putusan Mahkamah Agung Republik Indo...
1,Putusan PN MEDAN Nomor 1157/Pid.B/2024/PN Mdn ...,1157/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,22 Juli 2024,PN MEDAN,PN,Hakim Ketua Donald Panggabean,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan Terdakwa Erwin tersebut...,2 Oktober 2024,2 Oktober 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_1157_pid.b_2024_pn_mdn_20250625173554.pdf,Direktori Putusan Mahkamah Agung Republik Indo...
2,Putusan PN MEDAN Nomor 2065/Pid.B/2024/PN Mdn ...,2065/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,12 Nopember 2024,PN MEDAN,PN,Hakim Ketua Muhammad Kasim,...,PIDANA PENJARA WAKTU TERTENTU,MENGADILI: Menyatakan Terdakwa Joshua Corneliu...,19 Desember 2024,19 Desember 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_2065_pid.b_2024_pn_mdn_20250625173613.pdf,Direktori Putusan Mahkamah Agung Republik Indo...
3,Putusan PN MEDAN Nomor 826/Pid.B/2024/PN Mdn T...,826/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,3 Juni 2024,PN MEDAN,PN,Hakim Ketua Frans Effendi Manurung,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I : 1. Menyatakan Terdakwa I G...,18 Juli 2024,18 Juli 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,NaN,NaN,NaN
4,Putusan PN MEDAN Nomor 338/Pid.B/2024/PN Mdn T...,338/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,4 Maret 2024,PN MEDAN,PN,Hakim Ketua Khairulludin,...,PIDANA PENJARA WAKTU TERTENTU,M E N G A D I L I Menyatakan terdakwa M. SYAFR...,13 Mei 2024,13 Mei 2024,—,NaN,https://putusan3.mahkamahagung.go.id/direktori...,https://putusan3.mahkamahagung.go.id/direktori...,putusan_338_pid.b_2024_pn_mdn_20250625173625.pdf,Direktori Putusan Mahkamah Agung Republik Indo...


In [161]:
# Cek nama kolom
df.columns

Index(['judul', 'nomor', 'tingkat_proses', 'klasifikasi', 'kata_kunci',
       'tahun', 'tanggal_register', 'lembaga_peradilan',
       'jenis_lembaga_peradilan', 'hakim_ketua', 'hakim_anggota', 'panitera',
       'amar', 'amar_lainnya', 'catatan_amar', 'tanggal_musyawarah',
       'tanggal_dibacakan', 'kaidah', 'abstrak', 'link', 'link_pdf',
       'file_name_pdf', 'text_pdf'],
      dtype='object')

In [162]:
# Hapus duplikat
df = df.drop_duplicates()

In [163]:
# Hapus kolom yang tidak diperlukan
columns_to_drop = ['link', 'link_pdf', 'file_name_pdf', 'text_pdf',]  # sesuaikan jika ada
df = df.drop(columns=columns_to_drop, errors='ignore')

In [167]:
# Fungsi untuk membersihkan teks
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'm\s*e\s*n\s*g\s*a\s*d\s*i\s*l\s*i\s*:?', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.strip()

# Preprocessing pada kolom catatan_amar
df['catatan_amar'] = df['catatan_amar'].apply(preprocess_text)

In [171]:
# Fungsi ekstraksi pidana penjara
def extract_pidana_penjara(text):
    if pd.isna(text):
        return None

    pattern = r'(?:pidana penjara selama|pidana penjara masing-masing|pidana penjara masing masing|pidana penjara masingmasing)\s+(.*?)(?:\s+menetapkan|\s+memerintahkan|$)'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)

    return match.group(1).strip() if match else None

# Tambahkan kolom hasil ekstraksi pidana penjara
df['pidana_penjara'] = df['catatan_amar'].apply(extract_pidana_penjara)

# Tampilkan hasil akhir
df

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,hakim_anggota,panitera,amar,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,pidana_penjara
0,Putusan PN MEDAN Nomor 621/Pid.B/2024/PN Mdn T...,621/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,2 Mei 2024,PN MEDAN,PN,Hakim Ketua Lucas Sahabat Duha,Mhbr Hakim Anggota Mohammad Yusafrihardi Girs...,Panitera Pengganti: Potalfin Siregar,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa i m kandi safari dan terda...,23 Juli 2024,23 Juli 2024,—,NaN,None
1,Putusan PN MEDAN Nomor 1157/Pid.B/2024/PN Mdn ...,1157/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,22 Juli 2024,PN MEDAN,PN,Hakim Ketua Donald Panggabean,"Br Hakim Anggota Zufida Hanum, Hakim Anggota ...",Panitera Pengganti Berry Prima P.,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa erwin tersebut diatas terb...,2 Oktober 2024,2 Oktober 2024,—,NaN,2 dua tahun
2,Putusan PN MEDAN Nomor 2065/Pid.B/2024/PN Mdn ...,2065/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,12 Nopember 2024,PN MEDAN,PN,Hakim Ketua Muhammad Kasim,"Hakim Anggota Frans Effendi Manurung, Br Haki...",Panitera Pengganti: Emmy Siahaan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa joshua cornelius manik ali...,19 Desember 2024,19 Desember 2024,—,NaN,1 satu tahun
3,Putusan PN MEDAN Nomor 826/Pid.B/2024/PN Mdn T...,826/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,3 Juni 2024,PN MEDAN,PN,Hakim Ketua Frans Effendi Manurung,"Br Hakim Anggota M. Nazir, Hakim Anggota Len...",Panitera Pengganti Via Ramalia Tarigan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,1 menyatakan terdakwa i ganda satria simanjunt...,18 Juli 2024,18 Juli 2024,—,NaN,1 satu tahun dan 6 enam bulan penjara 3
4,Putusan PN MEDAN Nomor 338/Pid.B/2024/PN Mdn T...,338/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,4 Maret 2024,PN MEDAN,PN,Hakim Ketua Khairulludin,"Mhbr Hakim Anggota Khamozaro Waruwu, Hakim An...",Panitera Pengganti: Aryandi,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa m syafrizal alias sahrijal...,13 Mei 2024,13 Mei 2024,—,NaN,1 satu tahun dan 6 enam bulan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Putusan PN MEDAN Nomor 2173/Pid.B/2023/PN Mdn ...,2173/Pid.B/2023/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,26 Oktober 2023,PN MEDAN,PN,Hakim Ketua Fauzul Hamdi.,"Hakim Anggota Nelson Panjaitan, Br Hakim Angg...",Panitera Pengganti: Aryandi,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwaarif ramadhan als arif kebo...,9 Januari 2024,9 Januari 2024,—,NaN,1 satu tahun dan 6 enam bulan
106,Putusan PN MEDAN Nomor 2343/Pid.B/2023/PN Mdn ...,2343/Pid.B/2023/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,14 Nopember 2023,PN MEDAN,PN,Hakim Ketua Eti Astuti,"Hakim Anggota Fahren, Br Hakim Anggota Nurmiati",Panitera Pengganti: Ngatas Purba,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa terdakwa i deni aman gulo ...,4 Januari 2024,4 Januari 2024,—,NaN,selama terdakwa i deni aman gulo als denis gul...
107,Putusan PA SAROLANGUN Nomor 179/Pdt.G/2025/PA....,179/Pdt.G/2025/PA.Srl,Pertama,Perdata Agama \n Perdata Agama Perceraian,Cerai Gugat,2025,26 Mei 2025,PA SAROLANGUN,PA,Hakim Tunggal Taufiqur Rakhman Alhaq,Hakim Tunggal Taufiqur Rakhman Alhaq,Panitera Pengganti: Arsad,Lain-lain,DIKABULKAN,menyatakan tergugat yang telah dipanggil secar...,25 Juni 2025,25 Juni 2025,—,NaN,None
108,Putusan PA SAROLANGUN Nomor 177/Pdt.G/2025/PA....,177/Pdt.G/2025/PA.Srl,Pertama,Perdata Agama \n Perdata Agama Perceraian,Cerai Talak,2025,22 Mei 2025,PA SAROLANGUN,PA,Hakim Tunggal Taufiqur Rakhman Alhaq,Hakim Tunggal Taufiqur Rakhman Alhaq,Panitera Pengganti: Anita Kirana,Lai

In [175]:
# Fungsi ekstraksi keadaan terdakwa
def extract_keadaan(text):
    if pd.isna(text):
        return None

    pattern = r'(?:meyakinkan bersalah|meyakinkanbersalah|menyatakan bersalah|dinyatakan bersalah|meyakinkan terbukti bersalah)\s+(.*?)(?:\s+menjatuhkan pidana|$)'
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)

    return match.group(1).strip() if match else None

# Tambahkan kolom hasil ekstraksi keadaan
df['kategori_penganiayaan'] = df['catatan_amar'].apply(extract_keadaan)

# Tampilkan hasil akhir
df

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,...,panitera,amar,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,kaidah,abstrak,pidana_penjara,kategori_penganiayaan
0,Putusan PN MEDAN Nomor 621/Pid.B/2024/PN Mdn T...,621/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,2 Mei 2024,PN MEDAN,PN,Hakim Ketua Lucas Sahabat Duha,...,Panitera Pengganti: Potalfin Siregar,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa i m kandi safari dan terda...,23 Juli 2024,23 Juli 2024,—,NaN,None,melakukan tindak pidana dengan terangterangan ...
1,Putusan PN MEDAN Nomor 1157/Pid.B/2024/PN Mdn ...,1157/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,22 Juli 2024,PN MEDAN,PN,Hakim Ketua Donald Panggabean,...,Panitera Pengganti Berry Prima P.,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa erwin tersebut diatas terb...,2 Oktober 2024,2 Oktober 2024,—,NaN,2 dua tahun,melakukan tindak pidana secara bersamasama mel...
2,Putusan PN MEDAN Nomor 2065/Pid.B/2024/PN Mdn ...,2065/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,12 Nopember 2024,PN MEDAN,PN,Hakim Ketua Muhammad Kasim,...,Panitera Pengganti: Emmy Siahaan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa joshua cornelius manik ali...,19 Desember 2024,19 Desember 2024,—,NaN,1 satu tahun,melakukan tindak pidana penganiayaan sebagaima...
3,Putusan PN MEDAN Nomor 826/Pid.B/2024/PN Mdn T...,826/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,3 Juni 2024,PN MEDAN,PN,Hakim Ketua Frans Effendi Manurung,...,Panitera Pengganti Via Ramalia Tarigan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,1 menyatakan terdakwa i ganda satria simanjunt...,18 Juli 2024,18 Juli 2024,—,NaN,1 satu tahun dan 6 enam bulan penjara 3,melakukan tindak pidana kekerasan terhadap ora...
4,Putusan PN MEDAN Nomor 338/Pid.B/2024/PN Mdn T...,338/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,4 Maret 2024,PN MEDAN,PN,Hakim Ketua Khairulludin,...,Panitera Pengganti: Aryandi,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa m syafrizal alias sahrijal...,13 Mei 2024,13 Mei 2024,—,NaN,1 satu tahun dan 6 enam bulan,melakukan tindak pidana dengan terang terangan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Putusan PN MEDAN Nomor 2173/Pid.B/2023/PN Mdn ...,2173/Pid.B/2023/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,26 Oktober 2023,PN MEDAN,PN,Hakim Ketua Fauzul Hamdi.,...,Panitera Pengganti: Aryandi,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwaarif ramadhan als arif kebo...,9 Januari 2024,9 Januari 2024,—,NaN,1 satu tahun dan 6 enam bulan,melakukan tindak pidanapenganiayaansebagaimana...
106,Putusan PN MEDAN Nomor 2343/Pid.B/2023/PN Mdn ...,2343/Pid.B/2023/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,14 Nopember 2023,PN MEDAN,PN,Hakim Ketua Eti Astuti,...,Panitera Pengganti: Ngatas Purba,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa terdakwa i deni aman gulo ...,4 Januari 2024,4 Januari 2024,—,NaN,selama terdakwa i deni aman gulo als denis gul...,melakukan tindak pidana pengeroyokan sebagaima...
107,Putusan PA SAROLANGUN Nomor 179/Pdt.G/2025/PA....,179/Pdt.G/2025/PA.Srl,Pertama,Perdata Agama \n Perdata Agama Perceraian,Cerai Gugat,2025,26 Mei 2025,PA SAROLANGUN,PA,Hakim Tunggal Taufiqur Rakhman Alhaq,...,Panitera Pengganti: Arsad,Lain-lain,DIKABULKAN,menyatakan tergugat yang telah dipanggil secar...,25 Juni 2025,25 Juni 2025,—,NaN,None,None
108,Putusan PA SAROLANGUN Nomor 177/Pdt.G/2025/PA....,177/Pdt.G/2025/PA.Srl,Pertama,Perdata Agama \n Perdata Agama Perceraian,Cerai Talak,2025,22 Mei 2025,PA SAROLANGUN,PA,Hakim Tunggal Taufiqur Rakhman Alhaq,...,Panitera Pengganti: Anita Kirana,Lain-lain,DIKABULKAN,1 

In [176]:
df.columns

Index(['judul', 'nomor', 'tingkat_proses', 'klasifikasi', 'kata_kunci',
       'tahun', 'tanggal_register', 'lembaga_peradilan',
       'jenis_lembaga_peradilan', 'hakim_ketua', 'hakim_anggota', 'panitera',
       'amar', 'amar_lainnya', 'catatan_amar', 'tanggal_musyawarah',
       'tanggal_dibacakan', 'kaidah', 'abstrak', 'pidana_penjara',
       'kategori_penganiayaan'],
      dtype='object')

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 107 entries, 0 to 109
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   judul                    107 non-null    object 
 1   nomor                    107 non-null    object 
 2   tingkat_proses           107 non-null    object 
 3   klasifikasi              107 non-null    object 
 4   kata_kunci               107 non-null    object 
 5   tahun                    107 non-null    int64  
 6   tanggal_register         107 non-null    object 
 7   lembaga_peradilan        107 non-null    object 
 8   jenis_lembaga_peradilan  107 non-null    object 
 9   hakim_ketua              107 non-null    object 
 10  hakim_anggota            107 non-null    object 
 11  panitera                 107 non-null    object 
 12  amar                     107 non-null    object 
 13  amar_lainnya             107 non-null    object 
 14  catatan_amar             107 no

In [178]:
# Cek jumlah missing value per kolom ===
print(df.isnull().sum())

judul                        0
nomor                        0
tingkat_proses               0
klasifikasi                  0
kata_kunci                   0
tahun                        0
tanggal_register             0
lembaga_peradilan            0
jenis_lembaga_peradilan      0
hakim_ketua                  0
hakim_anggota                0
panitera                     0
amar                         0
amar_lainnya                 0
catatan_amar                 0
tanggal_musyawarah           0
tanggal_dibacakan            0
kaidah                       0
abstrak                    107
pidana_penjara              10
kategori_penganiayaan       12
dtype: int64


In [179]:
columns_to_drop = [
    'abstrak','kaidah'
]
df = df.drop(columns=columns_to_drop)

In [181]:
print(df.isnull().sum())

judul                       0
nomor                       0
tingkat_proses              0
klasifikasi                 0
kata_kunci                  0
tahun                       0
tanggal_register            0
lembaga_peradilan           0
jenis_lembaga_peradilan     0
hakim_ketua                 0
hakim_anggota               0
panitera                    0
amar                        0
amar_lainnya                0
catatan_amar                0
tanggal_musyawarah          0
tanggal_dibacakan           0
pidana_penjara             10
kategori_penganiayaan      12
dtype: int64


In [182]:
kolom_missing = ['pidana_penjara', 'kategori_penganiayaan']

In [183]:
df = df.dropna(subset=kolom_missing)

In [184]:
print(df.isnull().sum())

judul                      0
nomor                      0
tingkat_proses             0
klasifikasi                0
kata_kunci                 0
tahun                      0
tanggal_register           0
lembaga_peradilan          0
jenis_lembaga_peradilan    0
hakim_ketua                0
hakim_anggota              0
panitera                   0
amar                       0
amar_lainnya               0
catatan_amar               0
tanggal_musyawarah         0
tanggal_dibacakan          0
pidana_penjara             0
kategori_penganiayaan      0
dtype: int64


In [185]:
df.head(10)

,judul,nomor,tingkat_proses,klasifikasi,kata_kunci,tahun,tanggal_register,lembaga_peradilan,jenis_lembaga_peradilan,hakim_ketua,hakim_anggota,panitera,amar,amar_lainnya,catatan_amar,tanggal_musyawarah,tanggal_dibacakan,pidana_penjara,kategori_penganiayaan
1,Putusan PN MEDAN Nomor 1157/Pid.B/2024/PN Mdn ...,1157/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,22 Juli 2024,PN MEDAN,PN,Hakim Ketua Donald Panggabean,"Br Hakim Anggota Zufida Hanum, Hakim Anggota ...",Panitera Pengganti Berry Prima P.,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa erwin tersebut diatas terb...,2 Oktober 2024,2 Oktober 2024,2 dua tahun,melakukan tindak pidana secara bersamasama mel...
2,Putusan PN MEDAN Nomor 2065/Pid.B/2024/PN Mdn ...,2065/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,12 Nopember 2024,PN MEDAN,PN,Hakim Ketua Muhammad Kasim,"Hakim Anggota Frans Effendi Manurung, Br Haki...",Panitera Pengganti: Emmy Siahaan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa joshua cornelius manik ali...,19 Desember 2024,19 Desember 2024,1 satu tahun,melakukan tindak pidana penganiayaan sebagaima...
3,Putusan PN MEDAN Nomor 826/Pid.B/2024/PN Mdn T...,826/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,3 Juni 2024,PN MEDAN,PN,Hakim Ketua Frans Effendi Manurung,"Br Hakim Anggota M. Nazir, Hakim Anggota Len...",Panitera Pengganti Via Ramalia Tarigan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,1 menyatakan terdakwa i ganda satria simanjunt...,18 Juli 2024,18 Juli 2024,1 satu tahun dan 6 enam bulan penjara 3,melakukan tindak pidana kekerasan terhadap ora...
4,Putusan PN MEDAN Nomor 338/Pid.B/2024/PN Mdn T...,338/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,4 Maret 2024,PN MEDAN,PN,Hakim Ketua Khairulludin,"Mhbr Hakim Anggota Khamozaro Waruwu, Hakim An...",Panitera Pengganti: Aryandi,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa m syafrizal alias sahrijal...,13 Mei 2024,13 Mei 2024,1 satu tahun dan 6 enam bulan,melakukan tindak pidana dengan terang terangan...
5,Putusan PN MEDAN Nomor 1116/Pid.B/2024/PN Mdn ...,1116/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,16 Juli 2024,PN MEDAN,PN,Hakim Ketua Sarma Siregar,"Hakim Anggota Vera Yetti Magdalena, Br Hakim ...",Panitera Pengganti: Emmy Siahaan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa puja asmara alias puja ter...,1 Oktober 2024,1 Oktober 2024,7 tujuh bulan,melakukan tindak pidana turut serta melakukan ...
6,Putusan PN MEDAN Nomor 598/Pid.B/2024/PN Mdn T...,598/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,30 April 2024,PN MEDAN,PN,Hakim Ketua Khairulludin,"Mhbr Hakim Anggota Khamozaro Waruwu, Hakim An...",Panitera Pengganti Abdul Rahman Rangkuti,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,1 menyatakan terdakwa budi irawan barus terseb...,17 Juli 2024,17 Juli 2024,2 dua tahun dan 10 sepuluh bulan 3,melakukan tindak pidana dimuka umum secara ber...
7,Putusan PN MEDAN Nomor 1904/Pid.B/2024/PN Mdn ...,1904/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,23 Oktober 2024,PN MEDAN,PN,Hakim Ketua Sarma Siregar,"Br Hakim Anggota Muhammad Kasim, Hakim Anggot...",Panitera Pengganti: Emmy Siahaan,Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa misael s simanjuntak terse...,17 Desember 2024,17 Desember 2024,1satu tahun,melakukan tindak pidana penganiayaan yang meng...
8,Putusan PN MEDAN Nomor 255/Pid.B/2024/PN Mdn T...,255/Pid.B/2024/PN Mdn,Pertama,Pidana Umum \n Pidana Umum Penganiayaan,Penganiayaan,2024,19 Februari 2024,PN MEDAN,PN,Hakim Ketua Firza Andriansyah,"Br Hakim Anggota Martua Sagala, Hakim Anggota...","S.kom, Panitera Pengganti Rahmadan Syahputra",Lain-lain,PIDANA PENJARA WAKTU TERTENTU,menyatakan terdakwa abdul saman tersebut diata...,7 Mei 2024,7 Mei 2024,selama 10 sepuluh bulan,melakukan tind

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90 entries, 1 to 106
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   judul                    90 non-null     object
 1   nomor                    90 non-null     object
 2   tingkat_proses           90 non-null     object
 3   klasifikasi              90 non-null     object
 4   kata_kunci               90 non-null     object
 5   tahun                    90 non-null     int64 
 6   tanggal_register         90 non-null     object
 7   lembaga_peradilan        90 non-null     object
 8   jenis_lembaga_peradilan  90 non-null     object
 9   hakim_ketua              90 non-null     object
 10  hakim_anggota            90 non-null     object
 11  panitera                 90 non-null     object
 12  amar                     90 non-null     object
 13  amar_lainnya             90 non-null     object
 14  catatan_amar             90 non-null     object


In [186]:
# Simpan hasil ke file CSV di local folder
output_path = 'data/preprocessed/cleaned_putusan_hasil.csv'
df.to_csv(output_path, index=False)

print(f"File berhasil disimpan di {output_path}")

File berhasil disimpan di data/preprocessed/cleaned_putusan_hasil.csv
